In [116]:
import pandas as pd
import numpy as np
import re
import os

#if running with Kaggle
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/zillow-prize-1/properties_2017.csv
/kaggle/input/zillow-prize-1/sample_submission.csv
/kaggle/input/zillow-prize-1/zillow_data_dictionary.xlsx
/kaggle/input/zillow-prize-1/train_2016_v2.csv
/kaggle/input/zillow-prize-1/properties_2016.csv
/kaggle/input/zillow-prize-1/train_2017.csv


#### **Common Utility Functions**

In [117]:
def remove_rows_with_empty_columns(df):
    # Select all columns except the first one
    other_columns = df.iloc[:, 1:]
    
    # Create a boolean mask for rows where all columns except the first one are NaN
    rows_to_drop = other_columns.isna().all(axis=1)
    
    # Drop those rows from the DataFrame
    df_cleaned = df[~rows_to_drop]
    
    return df_cleaned

## (1) Convert Excel to CSVs ('zillow_data_dictionary.xlsx')

In [118]:
# Kavindu
#file_path = './data/zillow_data_dictionary.xlsx'

# Vishal
file_path = r"/kaggle/input/zillow-prize-1/zillow_data_dictionary.xlsx"

def format_column_name(column_name):
    column_name = re.sub(r'ID$', '', column_name)
    formatted_name = re.sub(r'(?<!^)(?=[A-Z])', '_', column_name).lower()
    formatted_name += '_df'
    return formatted_name

excel_data = pd.read_excel(file_path, sheet_name=None)

for sheet_name, data in excel_data.items():
    sheet_name = format_column_name(sheet_name)
    #Kavindu
    #csv_file_name = f"./inputs/{sheet_name}.csv"

    #Vishal
    csv_file_name = f"/kaggle/working/{sheet_name}.csv"
    data.to_csv(csv_file_name, index=False)
    print(f"Saved {sheet_name} as {csv_file_name}")

#Kavindu
#os.path.exists("./inputs/data_dictionary_df.csv") and os.remove("./inputs/data_dictionary_df.csv")

#Vishal
#os.path.exists(r"F:/Projects - Learn/BI-BigDataProject Data/data_dictionary_df.csv") and os.remove(r"F:/Projects - Learn/BI-BigDataProject Data/data_dictionary_df.csv")

Saved data _dictionary_df as /kaggle/working/data _dictionary_df.csv
Saved heating_or_system_type_df as /kaggle/working/heating_or_system_type_df.csv
Saved property_land_use_type_df as /kaggle/working/property_land_use_type_df.csv
Saved story_type_df as /kaggle/working/story_type_df.csv
Saved air_conditioning_type_df as /kaggle/working/air_conditioning_type_df.csv
Saved architectural_style_type_df as /kaggle/working/architectural_style_type_df.csv
Saved type_construction_type_df as /kaggle/working/type_construction_type_df.csv
Saved building_class_type_df as /kaggle/working/building_class_type_df.csv


## (2) Split Master CSV to Tables ('properties_2017.csv')

In [119]:
#Kavindu
# df = pd.read_csv('./data/properties_2017.csv')

#Vishal
df = pd.read_csv("/kaggle/input/zillow-prize-1/properties_2017.csv")

/tmp/ipykernel_36/343657259.py:5: DtypeWarning: Columns (49) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/kaggle/input/zillow-prize-1/properties_2017.csv")


In [120]:
df.head()

,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,10754147,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,9.0,2016.0,9.0,NaN,NaN,NaN,NaN
1,10759547,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,27516.0,2015.0,27516.0,NaN,NaN,NaN,NaN
2,10843547,NaN,NaN,NaN,0.0,0.0,5.0,NaN,NaN,NaN,...,1.0,NaN,660680.0,1434941.0,2016.0,774261.0,20800.37,NaN,NaN,NaN
3,10859147,NaN,NaN,NaN,0.0,0.0,3.0,6.0,NaN,NaN,...,1.0,NaN,580059.0,1174475.0,2016.0,594416.0,14557.57,NaN,NaN,NaN
4,10879947,NaN,NaN,NaN,0.0,0.0,4.0,NaN,NaN,NaN,...,1.0,NaN,196751.0,440101.0,2016.0,243350.0,5725.17,NaN,NaN,NaN


In [121]:
initial_size = df.shape
initial_size

(2985217, 58)

In [122]:
# Drop unwanted columns
def drop_columns(df, columns):
    return df.drop(columns, axis=1)

columns_to_drop = [
    'fips', 'numberofstories', 'typeconstructiontypeid', 'censustractandblock', 
    'rawcensustractandblock', 'architecturalstyletypeid', 'taxdelinquencyyear', 
    'storytypeid', 'regionidneighborhood', 'finishedsquarefeet50', 
    'finishedsquarefeet15', 'finishedsquarefeet13', 'finishedsquarefeet12', 
    'finishedsquarefeet6', 'finishedfloor1squarefeet', 'buildingclasstypeid', 
    'basementsqft','taxdelinquencyflag', 'numberofstories', 'fireplaceflag', 'propertyzoningdesc'
]

reduced_df = drop_columns(df, columns_to_drop)

In [123]:
reduced_df.shape

(2985217, 38)

In [124]:
# Function to fill nulls with 0
def fill_nulls_with_zero(df, columns):
    df[columns] = df[columns].fillna(0)
    return df

In [125]:
# Columns with nulls to fill with 0
columns_to_fill = [
    'decktypeid', #66 if present, 0 if no
    'fireplacecnt', #0 if no fireplace
    'garagecarcnt', #0 if no garage
    'garagetotalsqft', #no garage, no squarefeet
    'hashottuborspa', #0 if no hottub or spa
    'poolcnt', #0 if no pools
    'poolsizesum', #0 if no pool
    'pooltypeid10', #0 if no spa or hot tub
    'pooltypeid2', #0 if no Pool with Spa/Hot Tub
    'pooltypeid7', #0 if no Pool without hot tub
    'threequarterbathnbr', #0 if no 3/4 bathroom
    'yardbuildingsqft17', #0 if no patio in yard
    'yardbuildingsqft26' #0 if no shed in yard
]

# Check 'garagecarcnt' and 'garagetotalsqft'

In [126]:
#Check whether garage count null and garage squarefeet null matches
reduced_df['garagecarcnt'].isnull().sum()

2094209

In [127]:
reduced_df['garagetotalsqft'].isnull().sum()

2094209

In [128]:
# Identify the indexes where 'garagecarcnt' is null
garagecarcnt_null_indexes = reduced_df[reduced_df['garagecarcnt'].isnull()].index

# Identify the indexes where 'garagetotalsqft' is null
garagetotalsqft_null_indexes = reduced_df[reduced_df['garagetotalsqft'].isnull()].index

# Check if the indexes match
indexes_match = garagecarcnt_null_indexes.equals(garagetotalsqft_null_indexes)

print(f"Do the indexes of null values in 'garagecarcnt' and 'garagetotalsqft' match? {indexes_match}")

Do the indexes of null values in 'garagecarcnt' and 'garagetotalsqft' match? True


# Check 'airconditioningtypeid'

In [129]:
air_conditioning_csv = pd.read_csv('/kaggle/working/air_conditioning_type_df.csv')
air_conditioning_csv.head(13)

,AirConditioningTypeID,AirConditioningDesc
0,1,Central
1,2,Chilled Water
2,3,Evaporative Cooler
3,4,Geo Thermal
4,5,NaN
5,6,Other
6,7,Packaged AC Unit
7,8,Partial
8,9,Refrigeration
9,10,Ventilation


In [130]:
reduced_df['airconditioningtypeid'].value_counts(dropna=False)

airconditioningtypeid
NaN     2169855
1.0      744291
13.0      58487
5.0        8795
9.0        1905
11.0       1818
12.0         59
3.0           7
Name: count, dtype: int64

In [131]:
# Fill nulls in 'airconditioningtypeid' with 5
reduced_df['airconditioningtypeid'].fillna(5, inplace=True)

/tmp/ipykernel_36/3232255604.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  reduced_df['airconditioningtypeid'].fillna(5, inplace=True)


In [132]:
reduced_df['airconditioningtypeid'].value_counts(dropna=False)

airconditioningtypeid
5.0     2178650
1.0      744291
13.0      58487
9.0        1905
11.0       1818
12.0         59
3.0           7
Name: count, dtype: int64

# Check 'heatingorsystemtypeid'

In [133]:
air_conditioning_csv = pd.read_csv('/kaggle/working/heating_or_system_type_df.csv')
air_conditioning_csv.head(25)

,HeatingOrSystemTypeID,HeatingOrSystemDesc
0,1,Baseboard
1,2,Central
2,3,Coal
3,4,Convection
4,5,Electric
5,6,Forced air
6,7,Floor/Wall
7,8,Gas
8,9,Geo Thermal
9,10,Gravity


In [134]:
reduced_df['heatingorsystemtypeid'].value_counts(dropna=False)

heatingorsystemtypeid
2.0     1168610
NaN     1116053
7.0      646021
6.0       27481
24.0      21107
20.0       3630
13.0       1342
18.0        586
1.0         262
14.0         41
10.0         39
12.0         25
11.0         16
21.0          3
19.0          1
Name: count, dtype: int64

In [135]:
# Fill nulls in 'heatingorsystemtypeid' with 13
reduced_df['heatingorsystemtypeid'].fillna(13, inplace=True)

/tmp/ipykernel_36/4086307121.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  reduced_df['heatingorsystemtypeid'].fillna(13, inplace=True)


In [136]:
reduced_df['heatingorsystemtypeid'].value_counts(dropna=False)

heatingorsystemtypeid
2.0     1168610
13.0    1117395
7.0      646021
6.0       27481
24.0      21107
20.0       3630
18.0        586
1.0         262
14.0         41
10.0         39
12.0         25
11.0         16
21.0          3
19.0          1
Name: count, dtype: int64

# Check 'unitcnt'

In [137]:
reduced_df['unitcnt'].value_counts(dropna=False)

unitcnt
1.0      1783873
NaN      1004175
2.0       115778
4.0        39926
3.0        39893
          ...   
72.0           1
132.0          1
125.0          1
199.0          1
130.0          1
Name: count, Length: 155, dtype: int64

In [138]:
# Number of units the structure is built into (i.e. 2 = duplex, 3 = triplex, etc...)

# Assume nulls in 'unitcnt' as 1
reduced_df['unitcnt'].fillna(1, inplace=True)

/tmp/ipykernel_36/3757886605.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  reduced_df['unitcnt'].fillna(1, inplace=True)


# Check 'buildingqualitytypeid'

 Overall assessment of condition of the building from best (lowest) to worst (highest)

In [139]:
reduced_df['buildingqualitytypeid'].value_counts(dropna=False)

buildingqualitytypeid
NaN     1043822
6.0      561502
8.0      501925
4.0      448049
7.0      190244
5.0      107053
9.0       69428
11.0      28488
3.0       17858
10.0      10629
12.0       4123
1.0        1776
2.0         320
Name: count, dtype: int64

In [140]:
# Assume nulls in 'buildingqualitytypeid' with 6 (Average Condition)
reduced_df['buildingqualitytypeid'].fillna(6, inplace=True)

/tmp/ipykernel_36/2763073674.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  reduced_df['buildingqualitytypeid'].fillna(6, inplace=True)


In [141]:
reduced_df['buildingqualitytypeid'].value_counts(dropna=False)

buildingqualitytypeid
6.0     1605324
8.0      501925
4.0      448049
7.0      190244
5.0      107053
9.0       69428
11.0      28488
3.0       17858
10.0      10629
12.0       4123
1.0        1776
2.0         320
Name: count, dtype: int64

# Fill the rest of columns

In [142]:
df_filled = fill_nulls_with_zero(reduced_df, columns_to_fill)

# Check if pool columns' nulls match

In [143]:
# Check where poolcnt is 0, but any of the other 4 columns have non-zero values
inconsistent_count = df_filled[(df_filled['poolcnt'] == 0) & ((df_filled['poolsizesum'] != 0) | 
                                       (df_filled['pooltypeid10'] != 0) | 
                                       (df_filled['pooltypeid2'] != 0) | 
                                       (df_filled['pooltypeid7'] != 0))].shape[0]

# Print the rows that violate the condition
print("Count of inconsistent rows:", inconsistent_count)


Count of inconsistent rows: 17006


In [144]:
df_filled.shape

(2985217, 38)

In [145]:
# Remove rows where poolcnt is 0 and any of the other 3 columns have non-zero values
df_filled2 = df_filled[~((df_filled['poolcnt'] == 0) & ((df_filled['poolsizesum'] != 0) |  
                                          (df_filled['pooltypeid2'] != 0) | 
                                          (df_filled['pooltypeid7'] != 0)))]

# Output the shape of the cleaned DataFrame to verify
print("Number of rows after removing inconsistencies:", df_filled2.shape[0])


Number of rows after removing inconsistencies: 2985217


# Merge the Pool columns

In [146]:
df_filled2['pooltypeid10'].value_counts(dropna=False)

pooltypeid10
0.0    2968211
1.0      17006
Name: count, dtype: int64

In [147]:
df_filled2['pooltypeid2'].value_counts(dropna=False)

pooltypeid2
0.0    2952161
1.0      33056
Name: count, dtype: int64

In [148]:
df_filled2['pooltypeid7'].value_counts(dropna=False)

pooltypeid7
0.0    2479322
1.0     505895
Name: count, dtype: int64

In [149]:
# Check if more than one pool column has a value of 1
mask = (reduced_df[['pooltypeid10', 'pooltypeid2', 'pooltypeid7']] == 1).sum(axis=1) > 1

# Filter the DataFrame to get the rows that meet the condition
rows_with_multiple_pools = df[mask]

# Display the rows that contain the value 1 for more than one pool column
print(rows_with_multiple_pools)

Empty DataFrame
Columns: [parcelid, airconditioningtypeid, architecturalstyletypeid, basementsqft, bathroomcnt, bedroomcnt, buildingclasstypeid, buildingqualitytypeid, calculatedbathnbr, decktypeid, finishedfloor1squarefeet, calculatedfinishedsquarefeet, finishedsquarefeet12, finishedsquarefeet13, finishedsquarefeet15, finishedsquarefeet50, finishedsquarefeet6, fips, fireplacecnt, fullbathcnt, garagecarcnt, garagetotalsqft, hashottuborspa, heatingorsystemtypeid, latitude, longitude, lotsizesquarefeet, poolcnt, poolsizesum, pooltypeid10, pooltypeid2, pooltypeid7, propertycountylandusecode, propertylandusetypeid, propertyzoningdesc, rawcensustractandblock, regionidcity, regionidcounty, regionidneighborhood, regionidzip, roomcnt, storytypeid, threequarterbathnbr, typeconstructiontypeid, unitcnt, yardbuildingsqft17, yardbuildingsqft26, yearbuilt, numberofstories, fireplaceflag, structuretaxvaluedollarcnt, taxvaluedollarcnt, assessmentyear, landtaxvaluedollarcnt, taxamount, taxdelinquencyfl

In [150]:
df_filled2['pooltypeid'] = pd.NA

# Assign values to 'pooltypeid' based on 'pooltypeid10', 'pooltypeid2', and 'pooltypeid7'
df_filled2.loc[df_filled2['pooltypeid10'] == 1, 'pooltypeid'] = 10
df_filled2.loc[df_filled2['pooltypeid2'] == 1, 'pooltypeid'] = 2
df_filled2.loc[df_filled2['pooltypeid7'] == 1, 'pooltypeid'] = 7

# Drop the original 'pooltypeid10', 'pooltypeid2', and 'pooltypeid7' columns
df_filled2.drop(columns=['pooltypeid10', 'pooltypeid2', 'pooltypeid7'], inplace=True)

df_filled2['pooltypeid'].fillna(0, inplace=True)

/tmp/ipykernel_36/4219198944.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_filled2['pooltypeid'].fillna(0, inplace=True)
/tmp/ipykernel_36/4219198944.py:11: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_filled2['pooltypeid'].fillna(0, inplace=True)


In [152]:
df_filled2['pooltypeid'].value_counts(dropna=False)

pooltypeid
0     2429260
7      505895
2       33056
10      17006
Name: count, dtype: int64

In [153]:
df_filled2.columns

Index(['parcelid', 'airconditioningtypeid', 'bathroomcnt', 'bedroomcnt',
       'buildingqualitytypeid', 'calculatedbathnbr', 'decktypeid',
       'calculatedfinishedsquarefeet', 'fireplacecnt', 'fullbathcnt',
       'garagecarcnt', 'garagetotalsqft', 'hashottuborspa',
       'heatingorsystemtypeid', 'latitude', 'longitude', 'lotsizesquarefeet',
       'poolcnt', 'poolsizesum', 'propertycountylandusecode',
       'propertylandusetypeid', 'regionidcity', 'regionidcounty',
       'regionidzip', 'roomcnt', 'threequarterbathnbr', 'unitcnt',
       'yardbuildingsqft17', 'yardbuildingsqft26', 'yearbuilt',
       'structuretaxvaluedollarcnt', 'taxvaluedollarcnt', 'assessmentyear',
       'landtaxvaluedollarcnt', 'taxamount', 'pooltypeid'],
      dtype='object')

In [154]:
# Check for missing values
missing_percentage = round((df_filled2.isnull().sum() / df_filled2.shape[0]) * 100, 2)

print("Missing percentage for each column:\n", missing_percentage)

Missing percentage for each column:
 parcelid                        0.00
airconditioningtypeid           0.00
bathroomcnt                     0.10
bedroomcnt                      0.10
buildingqualitytypeid           0.00
calculatedbathnbr               3.92
decktypeid                      0.00
calculatedfinishedsquarefeet    1.51
fireplacecnt                    0.00
fullbathcnt                     3.92
garagecarcnt                    0.00
garagetotalsqft                 0.00
hashottuborspa                  0.00
heatingorsystemtypeid           0.00
latitude                        0.10
longitude                       0.10
lotsizesquarefeet               9.14
poolcnt                         0.00
poolsizesum                     0.00
propertycountylandusecode       0.10
propertylandusetypeid           0.10
regionidcity                    2.08
regionidcounty                  0.10
regionidzip                     0.43
roomcnt                         0.10
threequarterbathnbr             0.00
u

In [155]:
# Remove all rows with any missing values
DF = df_filled2.dropna()

In [156]:
# Check for missing values
missing_percentage = round((DF.isnull().sum() / DF.shape[0]) * 100, 2)

print("Missing percentage for each column:\n", missing_percentage)

Missing percentage for each column:
 parcelid                        0.0
airconditioningtypeid           0.0
bathroomcnt                     0.0
bedroomcnt                      0.0
buildingqualitytypeid           0.0
calculatedbathnbr               0.0
decktypeid                      0.0
calculatedfinishedsquarefeet    0.0
fireplacecnt                    0.0
fullbathcnt                     0.0
garagecarcnt                    0.0
garagetotalsqft                 0.0
hashottuborspa                  0.0
heatingorsystemtypeid           0.0
latitude                        0.0
longitude                       0.0
lotsizesquarefeet               0.0
poolcnt                         0.0
poolsizesum                     0.0
propertycountylandusecode       0.0
propertylandusetypeid           0.0
regionidcity                    0.0
regionidcounty                  0.0
regionidzip                     0.0
roomcnt                         0.0
threequarterbathnbr             0.0
unitcnt                    

In [157]:
DF.shape

(2610233, 36)

In [158]:
DF.head(20)

,parcelid,airconditioningtypeid,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,decktypeid,calculatedfinishedsquarefeet,fireplacecnt,fullbathcnt,...,unitcnt,yardbuildingsqft17,yardbuildingsqft26,yearbuilt,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,pooltypeid
20,11324547,1.0,2.0,4.0,8.0,2.0,0.0,3633.0,0.0,2.0,...,1.0,0.0,0.0,2005.0,222321.0,296425.0,2016.0,74104.0,6941.39,0
28,11488147,5.0,4.0,5.0,8.0,4.0,0.0,2865.0,0.0,4.0,...,2.0,0.0,0.0,1972.0,271704.0,831224.0,2016.0,559520.0,10455.41,0
30,11524947,1.0,2.0,2.0,7.0,2.0,0.0,1090.0,0.0,2.0,...,1.0,0.0,0.0,2004.0,232897.0,357568.0,2016.0,124671.0,6165.36,0
33,11585547,5.0,2.0,3.0,8.0,2.0,0.0,2077.0,0.0,2.0,...,1.0,0.0,0.0,1926.0,210192.0,646760.0,2016.0,436568.0,7924.68,0
45,11759947,5.0,2.0,4.0,4.0,2.0,0.0,1931.0,0.0,2.0,...,2.0,0.0,0.0,1957.0,124789.0,224617.0,2016.0,99828.0,2885.93,0
75,12144747,5.0,4.0,4.0,4.0,4.0,0.0,3086.0,0.0,4.0,...,2.0,0.0,0.0,1950.0,288506.0,820065.0,2016.0,531559.0,9602.77,0
96,12508747,5.0,3.0,0.0,6.0,3.0,0.0,9893.0,0.0,3.0,...,1.0,0.0,0.0,2004.0,1212818.0,1742687.0,2016.0,529869.0,21738.76,0
103,12606547,1.0,3.0,3.0,7.0,3.0,0.0,2166.0,0.0,3.0,...,1.0,0.0,0.0,2005.0,363000.0,659000.0,2016.0,296000.0,7580.62,0
110,12716947,5.0,1.0,3.0,4.0,1.0,0.0,1244.0,0.0,1.0,...,1.0,0.0,0.0,1950.0,108040.0,169471.0,2016.0,61431.0,2532.88,0
111,12757147,1.0,2.0,3.0,4.0,2.0,0.0,1300.0,0.0,2.0,...,1.0,0.0,0.0,1950.0,77415.0,233266.0,2016.0,155851.0,3110.99,0


In [159]:
DF.columns

Index(['parcelid', 'airconditioningtypeid', 'bathroomcnt', 'bedroomcnt',
       'buildingqualitytypeid', 'calculatedbathnbr', 'decktypeid',
       'calculatedfinishedsquarefeet', 'fireplacecnt', 'fullbathcnt',
       'garagecarcnt', 'garagetotalsqft', 'hashottuborspa',
       'heatingorsystemtypeid', 'latitude', 'longitude', 'lotsizesquarefeet',
       'poolcnt', 'poolsizesum', 'propertycountylandusecode',
       'propertylandusetypeid', 'regionidcity', 'regionidcounty',
       'regionidzip', 'roomcnt', 'threequarterbathnbr', 'unitcnt',
       'yardbuildingsqft17', 'yardbuildingsqft26', 'yearbuilt',
       'structuretaxvaluedollarcnt', 'taxvaluedollarcnt', 'assessmentyear',
       'landtaxvaluedollarcnt', 'taxamount', 'pooltypeid'],
      dtype='object')

# Defining the Tables

# 1. Property Table

In [160]:
property_columns = [
    'parcelid', 'latitude', 'longitude', 'lotsizesquarefeet', 'yearbuilt', 
    'roomcnt', 'unitcnt', 'calculatedfinishedsquarefeet', 'buildingqualitytypeid', 
    'propertylandusetypeid', 'propertycountylandusecode', 'regionidcity', 'regionidcounty', 'regionidzip'
]
property_table = DF[property_columns]

In [161]:
property_table.head()

,parcelid,latitude,longitude,lotsizesquarefeet,yearbuilt,roomcnt,unitcnt,calculatedfinishedsquarefeet,buildingqualitytypeid,propertylandusetypeid,propertycountylandusecode,regionidcity,regionidcounty,regionidzip
20,11324547,34560018.0,-118169806.0,9826.0,2005.0,0.0,1.0,3633.0,8.0,261.0,0100,40227.0,3101.0,97329.0
28,11488147,33879216.0,-118361434.0,4990.0,1972.0,0.0,2.0,2865.0,8.0,246.0,0200,33612.0,3101.0,96124.0
30,11524947,33974100.0,-118423000.0,40247.0,2004.0,0.0,1.0,1090.0,7.0,266.0,010C,12447.0,3101.0,96072.0
33,11585547,34012977.0,-118479243.0,6490.0,1926.0,0.0,1.0,2077.0,8.0,261.0,0100,26964.0,3101.0,96152.0
45,11759947,33990297.0,-118275168.0,5420.0,1957.0,0.0,2.0,1931.0,4.0,246.0,0200,12447.0,3101.0,96018.0


# 2. Building Table


In [162]:
building_columns = [
    'parcelid', 'bathroomcnt', 'bedroomcnt', 'fullbathcnt', 'calculatedbathnbr', 
    'threequarterbathnbr', 'fireplacecnt', 'garagecarcnt', 'garagetotalsqft', 
    'airconditioningtypeid', 'heatingorsystemtypeid', 'decktypeid'
]
building_table = DF[building_columns]

In [163]:
building_table.head()

,parcelid,bathroomcnt,bedroomcnt,fullbathcnt,calculatedbathnbr,threequarterbathnbr,fireplacecnt,garagecarcnt,garagetotalsqft,airconditioningtypeid,heatingorsystemtypeid,decktypeid
20,11324547,2.0,4.0,2.0,2.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0
28,11488147,4.0,5.0,4.0,4.0,0.0,0.0,0.0,0.0,5.0,13.0,0.0
30,11524947,2.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0
33,11585547,2.0,3.0,2.0,2.0,0.0,0.0,0.0,0.0,5.0,2.0,0.0
45,11759947,2.0,4.0,2.0,2.0,0.0,0.0,0.0,0.0,5.0,7.0,0.0


# 3. BuildingQuality Table

In [164]:
buildingquality_columns = {
    'buildingqualitytypeid': range(1, 13), 
    'quality_description': [
    "Best",
    "Excellent",
    "Very good",
    "Good",
    "Above average",
    "Average",
    "Below average",
    "Fair",
    "Poor",
    "Very poor",
    "Bad",
    "Worst"
]
}

building_quality_table = pd.DataFrame(buildingquality_columns)


In [165]:
building_quality_table.head(12)

,buildingqualitytypeid,quality_description
0,1,Best
1,2,Excellent
2,3,Very good
3,4,Good
4,5,Above average
5,6,Average
6,7,Below average
7,8,Fair
8,9,Poor
9,10,Very poor


# 4. AirConditioning Table

In [166]:
# Kavindu 
#csv_file_path = 

# Vishal
csv_file_path = "/kaggle/working/air_conditioning_type_df.csv"

air_conditioning_table = pd.read_csv(csv_file_path)
air_conditioning_table.columns = ['airconditioningtypeid', 'ac_type']

In [167]:
air_conditioning_table.head(13)

,airconditioningtypeid,ac_type
0,1,Central
1,2,Chilled Water
2,3,Evaporative Cooler
3,4,Geo Thermal
4,5,NaN
5,6,Other
6,7,Packaged AC Unit
7,8,Partial
8,9,Refrigeration
9,10,Ventilation


# 5. HeatingSystem Table

In [168]:
# Kavindu 
#csv_file_path = 

# Vishal
csv_file_path = "/kaggle/working/heating_or_system_type_df.csv"

heating_system_table = pd.read_csv(csv_file_path)
heating_system_table.columns = ['heatingsystemtypeid', 'hs_type']

In [169]:
heating_system_table.head(25)

,heatingsystemtypeid,hs_type
0,1,Baseboard
1,2,Central
2,3,Coal
3,4,Convection
4,5,Electric
5,6,Forced air
6,7,Floor/Wall
7,8,Gas
8,9,Geo Thermal
9,10,Gravity


# 6. Pool Type Table

In [170]:
pool_type_columns = {
    'pooltypeid': [10, 2, 7],
    'pool_type_description': [
        "Spa or Hot Tub",
        "Pool with Spa/Hot Tub",
        "Pool without hot tub"
    ]
}

pool_type_table = pd.DataFrame(pool_type_columns)

In [171]:
pool_type_table.head()

,pooltypeid,pool_type_description
0,10,Spa or Hot Tub
1,2,Pool with Spa/Hot Tub
2,7,Pool without hot tub


# 7. Pool Table

In [172]:
pool_columns = [
    'parcelid', 'poolcnt', 'poolsizesum', 'pooltypeid'
]
pool_table = DF[pool_columns]

In [175]:
pool_table.head()

,parcelid,poolcnt,poolsizesum,pooltypeid
20,11324547,0.0,0.0,0
28,11488147,0.0,0.0,0
30,11524947,0.0,0.0,0
33,11585547,0.0,0.0,0
45,11759947,0.0,0.0,0


# 8. Region Table

In [177]:
region_columns = ['parcelid','regionidcity', 'regionidcounty', 'regionidzip']
region_table = DF[region_columns]

In [178]:
region_table.head()

,parcelid,regionidcity,regionidcounty,regionidzip
20,11324547,40227.0,3101.0,97329.0
28,11488147,33612.0,3101.0,96124.0
30,11524947,12447.0,3101.0,96072.0
33,11585547,26964.0,3101.0,96152.0
45,11759947,12447.0,3101.0,96018.0


# 9. Tax Table

In [179]:
tax_columns = ['parcelid', 'taxvaluedollarcnt', 'structuretaxvaluedollarcnt',
    'landtaxvaluedollarcnt', 'taxamount', 'assessmentyear']
tax_table = DF[tax_columns]

In [180]:
tax_table.head()

,parcelid,taxvaluedollarcnt,structuretaxvaluedollarcnt,landtaxvaluedollarcnt,taxamount,assessmentyear
20,11324547,296425.0,222321.0,74104.0,6941.39,2016.0
28,11488147,831224.0,271704.0,559520.0,10455.41,2016.0
30,11524947,357568.0,232897.0,124671.0,6165.36,2016.0
33,11585547,646760.0,210192.0,436568.0,7924.68,2016.0
45,11759947,224617.0,124789.0,99828.0,2885.93,2016.0


# 10. Yard Table	

In [181]:
yard_columns = ['parcelid', 'yardbuildingsqft17', 'yardbuildingsqft26']
yard_table = DF[yard_columns]

In [182]:
yard_table.head()

,parcelid,yardbuildingsqft17,yardbuildingsqft26
20,11324547,0.0,0.0
28,11488147,0.0,0.0
30,11524947,0.0,0.0
33,11585547,0.0,0.0
45,11759947,0.0,0.0
